In [3]:
from pororo import Pororo
import numpy as np
from tqdm import tqdm

from datasets import load_dataset, load_from_disk

from transformers import (
    AutoTokenizer,
    AutoConfig,
)

import torch
from transformers import Trainer, TrainingArguments

mu, sigma = 0, 0.1
ner = Pororo(task="ner", lang="ko")

model_name = 'klue/roberta-large'

# set up tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

2023-01-05 08:14:33 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [input] dictionary: 4005 types
2023-01-05 08:14:33 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [label] dictionary: 41 types


In [4]:
data = load_from_disk('../dataset/train_dataset')
data

contexts = []
answers = []
for d in data['train']:
    contexts.append(d['context'])
    answers.append(d['answers']['text'][0])
    
print(len(contexts), len(answers))

for d in data['validation']:
    contexts.append(d['context'])
    answers.append(d['answers']['text'][0])
    
print(len(contexts), len(answers))

3952 3952
4192 4192


In [7]:
import re

i = 100
context = '동아대학교박물관에서 소장하고 있는 4 계사명 5개 사리구는 총 4개의 용기로 구성된 조선후기의 유물로,'
answer = '4개'

string = ''
labels = []
inputs = []
tokenized_text = tokenizer.tokenize(context)
tokenized_ids = tokenizer(context)['input_ids'][1:-1]
tokenized_ans = tokenizer.tokenize(answer)
n = 0
for text, ids in zip(tokenized_text, tokenized_ids):
    labels.append(ids)
    inputs.append(ids)
    add = text[2:] if '##' in text else ' '+text
    string += add
    
    if(text == tokenized_ans[n]):
        n += 1
        end = len(tokenized_ans)
        if(n == end):
            inputs = inputs[:-end] + tokenizer('[MASK]' * len(tokenized_ans))['input_ids'][1:-1]
            n = 0
    #else: n = 0
        
print(string)
print(labels)
print(inputs)
print(tokenizer.decode(inputs))
print(tokenizer.decode(labels))
print(len(labels), len(inputs))

 동아대학교박물관에서 소장하고 있는 4 계사명 5개 사리구는 총 4개의 용기로 구성된 조선후기의 유물로 ,
[5165, 2104, 4622, 2042, 2266, 2133, 27135, 5366, 19521, 1513, 2259, 24, 593, 2063, 2211, 25, 2019, 11955, 2251, 2259, 1668, 24, 2019, 2079, 6153, 2200, 3896, 2897, 3957, 2158, 2015, 2079, 9291, 2200, 16]
[5165, 2104, 4622, 2042, 2266, 2133, 27135, 5366, 19521, 1513, 2259, 24, 593, 2063, 2211, 4, 4, 11955, 2251, 2259, 1668, 4, 4, 2079, 6153, 2200, 3896, 2897, 3957, 2158, 2015, 2079, 9291, 2200, 16]
동아대학교박물관에서 소장하고 있는 4 계사명 [MASK] [MASK] 사리구는 총 [MASK] [MASK]의 용기로 구성된 조선후기의 유물로,
동아대학교박물관에서 소장하고 있는 4 계사명 5개 사리구는 총 4개의 용기로 구성된 조선후기의 유물로,
35 35


In [104]:
tokenizer.tokenize(tokenizer.decode(inputs))

['동아',
 '##대',
 '##학교',
 '##박',
 '##물',
 '##관',
 '##에서',
 '소장',
 '##하고',
 '있',
 '##는',
 '4',
 '계',
 '##사',
 '[MASK]',
 '[MASK]',
 '사리',
 '##구',
 '##는',
 '총',
 '[MASK]',
 '[MASK]',
 '의',
 '용기',
 '##로',
 '구성',
 '##된',
 '조선',
 '##후',
 '##기',
 '##의',
 '유물',
 '##로',
 ',']

In [100]:
def load_and_mask():
    data = load_from_disk('../dataset/train_dataset')
    
    contexts = []
    answers = []
    for d in data['train']:
        contexts.append(d['context'])
        answers.append(d['answers']['text'][0])
        
    for d in data['validation']:
        contexts.append(d['context'])
        answers.append(d['answers']['text'][0])
    
    wiki_contexts = []
    wiki_answers = []
    for t, a in zip(contexts, answers):
        while(len(t) > 500):
            wiki_contexts.append(t[:500])
            wiki_answers.append(a)
            t = t[384:]  # stride = 500 - 384 
    
    corpus = []
    for context, answer in zip(tqdm(wiki_contexts), wiki_answers):
        labels = []
        inputs = []
        tokenized_text = tokenizer.tokenize(context)
        tokenized_ids = tokenizer(context)['input_ids'][1:-1]
        tokenized_ans = tokenizer.tokenize(answer)
        
        n = 0
        for text, ids in zip(tokenized_text, tokenized_ids):
            labels.append(ids)
            inputs.append(ids)
            
            if(text == tokenized_ans[n]):
                n += 1
                end = len(tokenized_ans)
                if(n == end):
                    inputs = inputs[:-end] + tokenizer('[MASK]' * len(tokenized_ans))['input_ids'][1:-1]
                    n = 0
            else: n = 0
        
        corpus.append({"document_text": tokenizer.decode(labels), "masked_strings": tokenizer.decode(inputs), "inputs":inputs, "labels":labels})
        
    return corpus


In [79]:
corpus = load_and_mask()

100%|██████████| 6897/6897 [00:23<00:00, 298.25it/s]


In [101]:
import json

def make_data():
    with open("./dataset/wikipedia_documents_ssm_qa2.json", "w") as f:
        json.dump(load_and_mask(), f)
        #json.dump(corpus, f)

In [102]:
make_data()

100%|██████████| 6897/6897 [00:23<00:00, 298.55it/s]


In [8]:
from datasets import load_dataset, load_from_disk

a = load_from_disk('./embedding_train')
a

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'post', 'embedding'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'post', 'embedding'],
        num_rows: 240
    })
})

In [10]:
def load_train_dataset_emb(datasets, max_seq_length, tokenizer):
    print(datasets["train"])
    # dataset을 전처리합니다.
    # training과 evaluation에서 사용되는 전처리는 아주 조금 다른 형태를 가집니다.
    column_names = datasets["train"].column_names

    question_column_name = "question" if "question" in column_names else column_names[0]
    context_column_name = "post" if "post" in column_names else column_names[7]
    answer_column_name = "answers" if "answers" in column_names else column_names[2]
    # Padding에 대한 옵션을 설정합니다.
    # (question|context) 혹은 (context|question)로 세팅 가능합니다.
    pad_on_right = tokenizer.padding_side == "right"

    # Train preprocessing / 전처리를 진행합니다.
    def prepare_train_features(examples):  # example에는 dataset에 있는 item이 하나씩 들어갑니다. 
        # examples: [question, context, answers, title, id, document_id] -> 이 중 question, context, answers만 사용
        # truncation과 padding(length가 짧을때만)을 통해 toknization을 진행하며, stride를 이용하여 overflow를 유지합니다.
        # 각 example들은 이전의 context와 조금씩 겹치게됩니다.
        
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name], # 첫번째 문장(question)
            examples[context_column_name if pad_on_right else question_column_name], # 두번째 문장(context)
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            return_token_type_ids=False if 'roberta' in tokenizer.name_or_path else True, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length",
        )
        
        tokenized_examples_emb = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples['embedding'],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            return_token_type_ids=False if 'roberta' in tokenizer.name_or_path else True, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length",
        )
        
        for i,v in enumerate(tokenized_examples_emb["input_ids"]):
            tokenized_examples_emb["input_ids"][i] = [n if n==1 else 0 for n in v] 
        
        # 길이가 긴 context가 등장할 경우 truncate를 진행해야하므로, 해당 데이터셋을 찾을 수 있도록 mapping 가능한 값이 필요합니다.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # token의 캐릭터 단위 position를 찾을 수 있도록 offset mapping을 사용합니다.
        # start_positions과 end_positions을 찾는데 도움을 줄 수 있습니다.
        offset_mapping = tokenized_examples.pop("offset_mapping")

        # 데이터셋에 "start position", "enc position" label을 부여합니다.
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
        tokenized_examples["inputs_embeds"] = tokenized_examples_emb["input_ids"]

        for i, offsets in enumerate(offset_mapping):
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)  # cls index

            # sequence id를 설정합니다 (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # 하나의 example이 여러개의 span을 가질 수 있습니다.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]

            # answer가 없을 경우 cls_index를 answer로 설정합니다(== example에서 정답이 없는 경우 존재할 수 있음).
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # text에서 정답의 Start/end character index
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # text에서 current span의 Start token index
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1

                # text에서 current span의 End token index
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1

                # 정답이 span을 벗어났는지 확인합니다(정답이 없는 경우 CLS index로 label되어있음).
                if not (
                    offsets[token_start_index][0] <= start_char
                    and offsets[token_end_index][1] >= end_char
                ):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # token_start_index 및 token_end_index를 answer의 끝으로 이동합니다.
                    # Note: answer가 마지막 단어인 경우 last offset을 따라갈 수 있습니다(edge case).
                    while (
                        token_start_index < len(offsets)
                        and offsets[token_start_index][0] <= start_char
                    ):
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples

    if "train" not in datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = datasets["train"]

    # dataset에서 train feature를 생성합니다.
    train_dataset = train_dataset.map(
        prepare_train_features,
        batched=True,  # defalut batch size는 1000입니다.
        remove_columns=column_names,  
        # 기존 column_names인 ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__']를 삭제하고 ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']를 추가합니다.
        
    )
    print(train_dataset)
    return train_dataset


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
special_tokens_dict = {"additional_special_tokens": ["[MASK]", "\\n", "\\"]}
tokenizer.add_special_tokens(special_tokens_dict)

datasets = a
x = load_train_dataset_emb(datasets, 512, tokenizer)

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'post', 'embedding'],
    num_rows: 3952
})


100%|██████████| 4/4 [00:06<00:00,  1.69s/ba]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions', 'inputs_embeds'],
    num_rows: 5707
})


In [38]:
print(len(x[2]['input_ids']), len(x[2]['inputs_embeds']))

512 512
